# 神经网络回归-deepFM

In [ ]:
# 导入必要的工具包
import time
import numpy as np 
import pandas as pd 
import tensorflow as tf

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
Y_Standard = True
# 生成的结果文件编号
out_name = '%ssummit=0.1-fe-%s.csv'

# path  = '../data/new/'
path_train = '../data/new/train=0.1/'
path_test = '../data/new/test/'
summit_path = '../data/summit/'

## 1 数据准备

In [ ]:
train = pd.read_csv(path_train + 'train=0.1-fe.csv')
train.head()

In [ ]:
test = pd.read_csv(path_test + 'tap_fun_test-fe.csv')
test.head()

In [ ]:
# 从原始数据中分离输入特征x和输出y
target = 'prediction_pay_price'
id = 'user_id'
train_y = train[target].values
train_X = train.drop([target, id], axis = 1)

# test_id = test[id]
test_X = test.drop([id], axis = 1)

In [ ]:
X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(train_X, train_y, random_state=33, test_size=0.1)
print('train.shape=%s X_train_part.shape=%s X_test_part.shape=%s'
      %(train.shape, X_train_part.shape, X_val_part.shape))

## 2 数据标准化

In [ ]:
# 分别初始化对特征和目标值的标准化器
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
X_train_part = ss_X.fit_transform(X_train_part)
X_val_part = ss_X.transform(X_val_part)
test_X = ss_X.transform(test_X)

if Y_Standard:
    y_train_part = ss_y.fit_transform(y_train_part.reshape(-1, 1))
    y_val_part = ss_y.transform(y_val_part.reshape(-1, 1))

In [ ]:
# 标准化数据还原 
def inverse_StandardScaler(predict):
    if Y_Standard:
        predict = ss_y.inverse_transform(predict)
        print('predict = ', predict)
    return predict

#输出预测后的数据
def generate_summit(predict):
    testPredict = test.copy()
    testPredict[target] = predict
    testPredict = testPredict[[id,target]]
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

## 3. 训练模型

In [ ]:
# 定义网络结构
n_input = X_train_part.shape[1]
n_output = y_train_part.shape[1]
tf_x = tf.placeholder(tf.float32, [None,n_input])     # input x
tf_y = tf.placeholder(tf.float32, [None,n_output])     # input y

# neural network layers
# layer_dim = [100] #神经元数量=数据维度 
# layer_dim = [107, 10, 1] #添加隐层，调整神经元个数，看看效果 
# layer_dim = [107, 1] 
layer_dim = [107, 50, 1] 

active_fun = tf.nn.relu
# active_fun = tf.nn.relu6 #调整激活函数

for i, dim in enumerate(layer_dim):
    if i == 0: # input_layer
        hidden_layer = tf.layers.dense(tf_x, dim, active_fun) 
    elif i == (len(layer_dim) - 1):
        output_layer = tf.layers.dense(hidden_layer, dim) 
    else:
        hidden_layer = tf.layers.dense(hidden_layer, dim, active_fun)
#     hidden_layer = tf.layers.batch_normalization(hidden_layer) #加快收敛
#     hidden_layer = tf.layers.dropout(hidden_layer, rate=0.5) #防止过拟合

loss = tf.losses.mean_squared_error(tf_y, output_layer)   # compute cost 损失函数 
# 调整优化器和学习率 
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss)

In [ ]:
def model_validation(epoch):
    ''' 模型校验 '''
#     train_loss = sess.run(loss, {tf_x: X_train_part, 
#                                     tf_y: y_train_part})
    val_loss = sess.run(loss, {tf_x: X_val_part, 
                                    tf_y: y_val_part})
    if epoch % 10 == 0:
        print('---------------- epoch=%s  val_loss=%s' % (epoch+1, val_loss))
    
    return val_loss

In [ ]:
# 训练参数
n_sample = X_train_part.shape[0]
# batch_size = int(n_sample / 2)
batch_size = 41184
n_step = int(np.ceil(n_sample / batch_size))
n_epoch = 1000
early_stopping_max = 10
early_stopping_threshold = 0.35
print('n_sample=%s batch_size=%s n_step=%s n_epoch=%s' % (n_sample, batch_size, n_step, n_epoch))

In [ ]:
# 进行训练
sess = tf.Session()
sess.run(tf.global_variables_initializer())         # initialize var in graph
loss_train_arr = []
loss_val_arr = []
time_count = []
# losses = []
early_stopping_n = 0

for epoch in range(n_epoch):
    ticks = time.time()
#     print('=============== epoch=%s' % (epoch+1))
    for step in range(n_step):
        # train and net output
        i_from = step * batch_size
        i_to = (step+1) * batch_size
        if i_to > (n_sample - 1): #索引越界处理 
            i_to = (n_sample - 1)
        _, loss_v, pred = sess.run([train_op, loss, output_layer], {tf_x: X_train_part[i_from:i_to], 
                                                         tf_y: y_train_part[i_from:i_to]})
#         losses.append(loss_v)
#         if step % 5 == 0:
#             print('step=%s loss=%s' % (step+1, loss_v))
    mean_loss = loss_v # np.mean(losses)
    loss_train_arr.append(mean_loss)
    val_loss = model_validation(epoch)
    loss_val_arr.append(val_loss)
    ticks = np.round((time.time() - ticks) * 100) / 100
    time_count.append(ticks)
    if epoch % 10 == 0:
        print('%ss/epoch  loss=%s' % (ticks, mean_loss))
    if val_loss < early_stopping_threshold:
        early_stopping_n += 1
    if early_stopping_n > 10:
        break

time_total = np.sum(time_count)
print('time_total=%ss time_epoch=%s'%(time_total, time_count[0]))

## 4. 训练结果可视化

In [ ]:
# 训练次数，x 轴
# x = np.arange(0, n_epoch, 1)
x = np.arange(0, len(loss_val_arr), 1)

In [ ]:
plt.figure(figsize=(18, 6))
# plt.plot(x, loss_train_arr, '-o', label='loss_train')
plt.plot(x, loss_train_arr, '-', label='loss_train')
plt.plot(x, loss_val_arr, '-', label='loss_val')

plt.xlabel('n_epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

## 5. 模型应用

In [ ]:
#对测试数据进行预测
test_pred = sess.run(output_layer, {tf_x:test_X})
print('test_pred = ', test_pred)

# 标准化数据还原 
test_pred = inverse_StandardScaler(test_pred)

In [ ]:
#输出预测后的数据
testPredict = generate_summit(test_pred)
testPredict.to_csv(out_name %(summit_path, 'DNN_V3'), index=False)
testPredict.head()

In [ ]:
# # 最后要关闭 
# sess.close()